In [4]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [5]:
from Hybrid import *
from Recommenders.SLIM.SLIMElasticNetRecommender import *
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
## SLIM
slim = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
slim.fit(topK=8894, l1_ratio=0.05565733019999427, alpha=0.0012979360257937668)
## PURESVD
pure_svd = PureSVDRecommender(URM_train)
pure_svd.fit(num_factors=70, random_seed=42)
## P3ALPHA
p3alpha = P3alphaRecommender(URM_train)
p3alpha.fit(topK=76, alpha=0.377201600381895, normalize_similarity=True)
## RP3BETA
rp3beta = RP3betaRecommender(URM_train)
rp3beta.fit(topK=101, alpha=0.3026342852596128, beta=0.058468783118329024)
## USER KNN
userknn = UserKNNCFRecommender(URM_train)
userknn.fit(topK=529, shrink=45, similarity='asymmetric', normalize=True,
                                       feature_weighting='TF-IDF')
## ITEM KNN
itemknn = ItemKNNCFRecommender(URM_train)
itemknn.fit(topK=31, shrink=435, similarity='tversky', normalize=True,
                                       feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)
## EASE_R
ease_r = EASE_R_Recommender(URM_train)
ease_r.fit(topK=24, l2_norm=37.54323189430143)

model = LinearHybridRecommender

from Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

SLIMElasticNetRecommender: URM Detected 451 ( 3.6%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 315 ( 1.4%) items with no interactions.


100%|█████████▉| 22216/22222 [01:34<00:00, 235.74it/s]


PureSVDRecommender: URM Detected 451 ( 3.6%) users with no interactions.
PureSVDRecommender: URM Detected 315 ( 1.4%) items with no interactions.
PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 0.57 sec
P3alphaRecommender: URM Detected 451 ( 3.6%) users with no interactions.
P3alphaRecommender: URM Detected 315 ( 1.4%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 9041.58 column/sec. Elapsed time 2.46 sec
RP3betaRecommender: URM Detected 451 ( 3.6%) users with no interactions.
RP3betaRecommender: URM Detected 315 ( 1.4%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 7839.76 column/sec. Elapsed time 2.83 sec
UserKNNCFRecommender: URM Detected 451 ( 3.6%) users with no interactions.
UserKNNCFRecommender: URM Detected 315 ( 1.4%) items with no interactions.
Similarity column 12638 (100.0%), 12405.01 column/sec. Elapsed time 1.02 sec
ItemKNNCFRecommender: UR

In [6]:
import optuna as op
def objective(trial):
    
    alphas = [trial.suggest_float("alphas[0]", 0, 2), trial.suggest_float("alphas[1]", 0, 2), trial.suggest_float("alphas[2]", 0, 2), trial.suggest_float("alphas[3]", 0, 2), trial.suggest_float("alphas[4]", 0, 2), trial.suggest_float("alphas[5]", 0, 2), trial.suggest_float("alphas[6]", 0, 2)]
    
    recommender = model(URM_train, [slim, pure_svd, p3alpha, rp3beta, userknn, itemknn, ease_r])
    #recommender.fit(**trial.params)
    recommender.fit(alphas)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [ ]:
best_params = {
    "alphas": 1,
}

study_name = "hybrid-allmodels-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db".format(study_name)
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
#study.enqueue_trial(best_params)
study.optimize(objective, n_trials=50)

In [ ]:
study.best_params

In [18]:
final = model(URM_train_validation, [slim, pure_svd, p3alpha, rp3beta, userknn, itemknn, ease_r])
final.fit(alphas=[study.best_params["alphas[0]"],study.best_params["alphas[1]"], study.best_params["alphas[2]"], study.best_params["alphas[3]"], study.best_params["alphas[4]"], study.best_params["alphas[5]"], study.best_params["alphas[6]"]])

SLIMElasticNetPureSVDP3alphaRP3betaUserKNNCFItemKNNCFEASE_R_HybridRecommender: URM Detected 218 ( 1.7%) users with no interactions.
SLIMElasticNetPureSVDP3alphaRP3betaUserKNNCFItemKNNCFEASE_R_HybridRecommender: URM Detected 113 ( 0.5%) items with no interactions.


In [19]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 2237 (17.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10401 (100.0%) in 9.20 sec. Users per second: 1130


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10       0.08752                 0.149037  0.124922  0.045067    0.074636   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.254793  0.137116  0.102929  0.489184      0.335242  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.822994          0.402595    0.822994       0.054405   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10           10.114115                   0.997897             0.155943   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   

In [20]:
from Recommenders.NonPersonalizedRecommender import TopPop
final = model(URM_all, [slim, pure_svd, p3alpha, rp3beta, userknn, itemknn, ease_r])
final.fit(alphas=[study.best_params["alphas[0]"],study.best_params["alphas[1]"], study.best_params["alphas[2]"], study.best_params["alphas[3]"], study.best_params["alphas[4]"], study.best_params["alphas[5]"], study.best_params["alphas[6]"]])
top_recommender = TopPop(URM_all)
top_recommender.fit()
generateSubmission(final, top_recommender)

/Users/alessandro/RecSys-2023-polimi/Data_manager/UserUtils.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['item_list'] = submission['item_list'].apply(lambda x: ' '.join(map(str, x)))  # remove brakets
/Users/alessandro/RecSys-2023-polimi/Data_manager/UserUtils.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['item_list'] = submission['item_list'].astype(str)
